In [4]:
import time
import scipy
import pickle
import scipy.io
import numpy as np
from scipy.linalg import norm
import matplotlib.pyplot as plt

In [55]:
data = scipy.io.loadmat('./data/syn_1000_1500_10_50.mat')
W = data['W']
with open('./data/1000_1500_sol.pckl', 'rb') as f:
    sol = pickle.load(f)

In [56]:
def obj_func(x):
    return -np.sum(np.log(W.dot(x)))

In [57]:
n, p = W.shape
x = np.ones(p) / p
n_steps = 1000
printst = 1
terminate_tol = 1e-8
sub_tol = 0.25 * terminate_tol
steps_tol = 0.05
Lest = 1

In [58]:
s_deactive = 0

if Lest == 1:
    #Estimate Lipschitz Constant
    Denom = W.dot(np.ones(p)) / p
    RatH = 1 / (Denom**2)    
    dirr = np.ones(p)
    for Liter in range(1, 16):
        Dir = W.T.dot(RatH * (W.dot(dirr)))
        dirr = Dir / norm(Dir)
    Hd = W.T.dot(RatH * (W.dot(dirr)))
    dHd = dirr.dot(Hd)
    L = dHd / (dirr.dot(dirr))

In [59]:
def fn_portf_proxsplx(y):
    m = len(y)
    bget = False
    s  = sorted(y, reverse=True)
    tmpsum = 0
    for i in range(m - 1):
        tmpsum = tmpsum + s[i]
        tmax = (tmpsum - 1) / (i + 1)
        if tmax >= s[i+1]:
            bget = True
            break
    if not bget:
        tmax = (tmpsum + s[m - 1] - 1) / m
    x = np.maximum(y - tmax, 0)
    return x

In [60]:
def fn_portf_fista(Grad, Hopr, x, L, tol):
    y = x.copy()
    x_cur = y.copy()
    t = 1
    kmax = 1200
    for k in range(1, kmax + 1):
        DQ = Hopr(y - x) + Grad
        x_tmp = y - 1 / L * DQ
        x_nxt = fn_portf_proxsplx(x_tmp)
        xdiff = x_nxt - x_cur
        ndiff = norm(xdiff)
        if (ndiff < tol) and (k > 1):
            print('Fista err = %3.3e; Subiter = %3d; subproblem converged!\n' % (ndiff, k))
            break
        t_nxt = 0.5 * (1 + np.sqrt(1 + 4 * t**2))
        y = x_nxt + (t - 1) / t_nxt * xdiff
        t  = t_nxt
        x_cur = x_nxt
    return x_nxt

In [61]:
err, times, points = [], [], []

points.append(x)

int_start = time.time()
for i in range(1, n_steps + 1):
    
    start = time.time()
    
    # Compute the denominator.
    denom = W.dot(x) # n by 1 vector
    
    # Evaluate the gradient.
    ratG = 1 / denom # n by 1 vector
    Grad = -W.T.dot(ratG) # p by 1 vector
    
    # Evaluate the Hessian
    ratH = 1 / (denom**2) # n by 1 vector
    Hopr = lambda d: W.T.dot(ratH * (W.dot(d))) # p by 1 vector
 
    if Lest == 0:
        # Compute Lipschitz Constant  
        dirr = np.ones(p)
        for Liter in range(1, 21):
            Dir = Hopr(dirr)
            dirr = Dir / norm(Dir)
        Hd = Hopr(dirr)
        dHd  = dirr.dot(Hd)
        L = dHd / (dirr.dot(dirr))
    
    x_nxt = fn_portf_fista(Grad, Hopr, x, L, sub_tol)
    diffx = x_nxt - x
    
    # solution value stop-criterion    
    nrm_dx = norm(diffx)
    rdiff = nrm_dx / max(1.0, norm(x))
    err.append(rdiff)
    
    # Check the stopping criterion.
    if (rdiff <= terminate_tol) and i > 1:
        print('Convergence achieved!')
        print('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n' % (i, s, rdiff))
        break
    
    # Compute a step-size if required.
    if not s_deactive:
        Hd = Hopr(diffx)
        dHd = diffx.dot(Hd)
        lams = np.sqrt(dHd)
        s = 1 / (1 + lams)   # 0.5 * Mf = 1
        
    if (1 - s <= steps_tol):
        s = 1
        s_deactive = 1               
    x = x + s * diffx
    
    end = time.time()
    
    times.append(end - start)
    points.append(x)
    
    if (i % printst == 0) or (i == 1):
        print('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n' % (i, s, rdiff))

# if mod(iter, options.printst) ~= 0
#     fprintf('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n', iter, s, rdiff);
# end

int_end = time.time()
if i >= n_steps:
    print('Exceed the maximum number of iterations')

iter =    1, stepsize = 6.693e-01, rdiff = 3.020e-01

iter =    2, stepsize = 8.164e-01, rdiff = 1.415e-01

iter =    3, stepsize = 8.925e-01, rdiff = 7.755e-02

iter =    4, stepsize = 9.248e-01, rdiff = 5.272e-02

iter =    5, stepsize = 9.450e-01, rdiff = 3.727e-02

iter =    6, stepsize = 1.000e+00, rdiff = 2.783e-02

iter =    7, stepsize = 1.000e+00, rdiff = 2.043e-02

iter =    8, stepsize = 1.000e+00, rdiff = 1.418e-02

iter =    9, stepsize = 1.000e+00, rdiff = 1.008e-02

iter =   10, stepsize = 1.000e+00, rdiff = 6.094e-03

iter =   11, stepsize = 1.000e+00, rdiff = 4.531e-03

iter =   12, stepsize = 1.000e+00, rdiff = 3.386e-03

iter =   13, stepsize = 1.000e+00, rdiff = 2.530e-03

iter =   14, stepsize = 1.000e+00, rdiff = 1.891e-03

iter =   15, stepsize = 1.000e+00, rdiff = 1.414e-03

iter =   16, stepsize = 1.000e+00, rdiff = 1.057e-03

iter =   17, stepsize = 1.000e+00, rdiff = 7.907e-04

iter =   18, stepsize = 1.000e+00, rdiff = 5.913e-04

iter =   19, stepsize = 1.00

In [62]:
with open('./concord_times.pckl', 'wb') as f:
    pickle.dump(times, f)

In [63]:
norm(x - sol) / max(1, norm(sol))

0.0012938336311932035

In [64]:
int_end - int_start

136.99677872657776

In [67]:
i

20